# Lugares turísticos BBDD turismoasturias.es

Extracción de nuevos datos de lugares turísticos a partir de bases de datos de Asturias.

In [26]:
import pandas as pd
import os
from unidecode import unidecode

In [27]:
def adaptar_caracteristicas(df, tipo_lugar, columnas, localidad, informacion):
    """
    Función para adaptar los nuevos lugares turisticos al formato de los obtenidos por web-scraping.
    
    Parámetros:
    - df: dataframe con los nuevos datos
    - tipo_lugar: tipo de lugar turístico al que corresponden esos datos.
    - columnas: columnas de los datos que consideraremos como caracteísticas
    - localidad: nombre de la variable que proporciona la "Localización"
    - informacion: tipo de campos que proporcionan la "Descripción" 
    
    Salida:
    - df_nuevo: dataframe con el formato adecuado de los datos.
    """

    # Inicializar el nuevo dataframe y las variables
    df_nuevo = pd.DataFrame()

    df_nuevo["Nombre"] = df["Nombre"]
    df_nuevo["Localización"] = df[localidad]
    df_nuevo["Latitud"] = None
    df_nuevo["Longitud"] = None
    df_nuevo["Características"] = None
    df_nuevo["Descripción"] = None
    df_nuevo["Tipo_lugar_turistico"] = tipo_lugar
    df_nuevo["url"] = None

    # Determinar las columnas que se incluiran en la "Descripción" 
    columnas_descripcion = [col for col in df.columns if informacion in unidecode(col.lower())]

    # Recorrer los datos para extraer las coordenadas, la descripción y la lista de características.
    for index in range(df.shape[0]):
        try:
            df_nuevo.loc[index, "Latitud"], df_nuevo.loc[index, "Longitud"] = df.loc[index, "Coordenadas"].split(",")
        except:
           pass

        df_nuevo['Descripción'] = df[columnas_descripcion].apply(lambda x: ' '.join(x.dropna()) if x.notna().all() and all(x != '') else x.dropna().iloc[0] if not x.dropna().empty else '', axis=1)

        df_nuevo['Características'] = df.apply(lambda row: [f"{col}: {row[col]}" for col in columnas if pd.notna(row[col])], axis=1)


    return df_nuevo

In [28]:
# Extraer todos los csv disponibles
ruta_archivos = '../Data/BBDD_asturias_tourism'

dfs = {} # Diccionario para almacenar los DataFrames con sus respectivos nombres

# Iterar sobre los archivos CSV en la carpeta
for archivo in os.listdir(ruta_archivos):
    if archivo.endswith('.csv'):
        ruta_completa = os.path.join(ruta_archivos, archivo) # Crear la ruta completa del archivo
        
        try:
            df = pd.read_csv(ruta_completa, encoding='ISO-8859-1', sep=';') # Leer el archivo
            nombre_archivo = os.path.splitext(archivo)[0] # Guardar el DataFrame con el nombre del archivo (sin .csv)
            dfs[nombre_archivo] = df
            print(f"Archivo '{archivo}' cargado correctamente.")
        except Exception as e:
            print(f"Error al cargar el archivo {archivo}: {e}")

Archivo 'Areas_autocaravanas.csv' cargado correctamente.
Archivo 'Bodegas_llagares_queserias.csv' cargado correctamente.
Archivo 'Espacios_naturales.csv' cargado correctamente.
Archivo 'Fiestas_interes_turistico.csv' cargado correctamente.
Archivo 'Museos_espacios_culturales.csv' cargado correctamente.
Archivo 'Oficinas_turismo.csv' cargado correctamente.
Archivo 'Patrimonio_religioso.csv' cargado correctamente.
Archivo 'Playas.csv' cargado correctamente.
Archivo 'Prerromanico.csv' cargado correctamente.
Archivo 'Pueblos_marineros.csv' cargado correctamente.
Archivo 'Rutas.csv' cargado correctamente.


In [29]:
# Retocar los valores del dataframe para evitar errores
for nombre, df in dfs.items():
    dfs[nombre] = df.apply(lambda col: col.map(lambda x: x.replace("??", "ll") if isinstance(x, str) else x))

Comparamos cada uno de los dataframes con los datos obtenidos por web-scraping, tratando de buscar nuevos lugares turísticos que podamos incorporar a nuestros datos

In [30]:
df_completo = pd.read_csv("../Data/Data_used/tourist_places_data.csv")

for nombre in dfs.keys():
    df_aux = dfs[nombre]

    nombres_df1 = set(df_aux["Nombre"].dropna())  # Eliminar NaN y convertir en conjunto
    nombres_df2 = set(df_completo["Nombre"].dropna())

    # Nombres que están en df1 pero no en df2
    nombres_solo_df1 = nombres_df1 - nombres_df2

    # Nombres que están en df2 pero no en df1
    nombres_solo_df2 = nombres_df2 - nombres_df1


    nombres_solo_df1 = list(nombres_solo_df1)
    nombres_solo_df2 = list(nombres_solo_df2)

    if len(nombres_solo_df1) == df_aux.shape[0]:
        print(f"{nombre}: tiene {df_aux.shape} elementos y {len(nombres_solo_df1)} nuevos --- todo")
    elif len(nombres_solo_df1) == 0:
        print(f"{nombre}: tiene {df_aux.shape} elementos y {len(nombres_solo_df1)} nuevos --- nada")
    else:
        print(f"{nombre}: tiene {df_aux.shape} elementos y {len(nombres_solo_df1)} nuevos --- comprobar")


Areas_autocaravanas: tiene (63, 21) elementos y 63 nuevos --- todo
Bodegas_llagares_queserias: tiene (50, 28) elementos y 0 nuevos --- nada
Espacios_naturales: tiene (206, 44) elementos y 0 nuevos --- nada
Fiestas_interes_turistico: tiene (89, 22) elementos y 89 nuevos --- todo
Museos_espacios_culturales: tiene (151, 39) elementos y 0 nuevos --- nada
Oficinas_turismo: tiene (67, 37) elementos y 67 nuevos --- todo
Patrimonio_religioso: tiene (56, 32) elementos y 0 nuevos --- nada
Playas: tiene (116, 27) elementos y 0 nuevos --- nada
Prerromanico: tiene (15, 31) elementos y 0 nuevos --- nada
Pueblos_marineros: tiene (18, 26) elementos y 1 nuevos --- comprobar
Rutas: tiene (228, 40) elementos y 23 nuevos --- comprobar


Nos centramos en los lugares turísticos (todo menos areas de autocaravanas y oficinas de turismo)

In [31]:
bodegas = dfs["Bodegas_llagares_queserias"]
print(bodegas.shape)
columnas = ["Dirección", "Teléfono", "Email", "Facebook", "Horario y duración de las visitas", "Tarifas", "Visitas guiadas"] # Columnas que se consideran importantes.
df_bodegas = adaptar_caracteristicas(bodegas, "Bodegas, llagares y queserías", columnas ,"Concejo", "descripcion")
df_bodegas.head()

(50, 28)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
0,Llagar Casa Trabanco,Gijón,43.470868,-5.645769,"[Dirección: Ctra. de Lavandera, 3255. 33350. L...",Si algo nos define en Casa Trabanco es nuestra...,"Bodegas, llagares y queserías",None
1,Bodegas Vitheras S.L.,Cangas del Narcea,43.119135,-6.489889,[Dirección: Polígono Industrial Obanca Nave D....,Viticultores Heroicos Asturianos S.L. (Vithera...,"Bodegas, llagares y queserías",None
2,Llagar Herminio,Oviedo,43.37716,-5.80400,"[Dirección: Camino Real, 11. 33010. Colloto/Cu...",Visitas guiadas a llagar de sidra Visita por l...,"Bodegas, llagares y queserías",None
3,Llagar Sidra Bernueces,Gijón,43.5089256,-5.6205887,"[Dirección: Camino de la Nisal, 1104. 33394. C...",Sidrería-restaurante que elabora su propia sid...,"Bodegas, llagares y queserías",None
4,Quesería Vega de Tordín,Cabrales,43.302437,-4.808721,"[Dirección: Camino de la Ería. 33554. Arenas, ...",En una visita guiada a la Quesería-Ganadería V...,"Bodegas, llagares y queserías",None


In [32]:
espacios_nat = dfs["Espacios_naturales"]
espacios_nat = espacios_nat[~espacios_nat["Nombre"].isin(df_completo["Nombre"])].reset_index(drop = True) # Eliminar los que ya están en los lugares turísticos
print(espacios_nat.shape)
columnas = ['Datos de Contacto', 'Servicios', "Observaciones"] # Columnas que se consideran importantes.
df_esp_nat = adaptar_caracteristicas(espacios_nat, "Naturaleza", columnas ,"Concejo", "texto")
df_esp_nat.head()

(0, 44)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url


In [33]:
museos = dfs["Museos_espacios_culturales"]
museos = museos[~museos["Nombre"].isin(df_completo["Nombre"])].reset_index(drop = True)
print(museos.shape)
columnas = ['Dirección', 'Código postal', 'Email', 'Web', 'Facebook', 'Horario', 'Tarifas'] # Columnas que se consideran importantes.
df_cultura = adaptar_caracteristicas(museos, "Cultura", columnas ,"Localidad", "texto")
df_cultura.head()

(0, 39)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url


In [34]:
religioso = dfs["Patrimonio_religioso"]
religioso = religioso[~religioso["Nombre"].isin(df_completo["Nombre"])].reset_index(drop = True)
print(religioso.shape)
columnas = ['Dirección', 'Código postal', 'Horario', 'Tarifas', 'Teléfono', 'Email', 'Web', "Época"] # Columnas que se consideran importantes.
df_religioso = adaptar_caracteristicas(religioso, "Patrimonio religioso", columnas ,"Localidad", "texto")
df_religioso.head()

(0, 32)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url


In [35]:
rutas = dfs["Rutas"]
rutas = rutas[rutas["TipoRuta"] != "coche"] # Eliminar las rutas en coche porque son de varios días y no hay coordenadas
rutas = rutas[~rutas["Nombre"].isin(df_completo["Nombre"])].reset_index(drop = True)
rutas["TipoRuta"] = rutas["TipoRuta"].str.lower()
print(rutas.shape)

# Cada tipo de ruta tiene una variable tiempo, las unificamos
rutas['Tiempo'] = rutas[['Tiempo a pie', 'Tiempo BTT', 'Tiempo en coche', 'Tiempo ascensión']].bfill(axis=1).iloc[:, 0]

columnas = ['Distancia', 'Dificultad', "Código", 'Tipo de recorrido', 'Altitud', 'Desnivel', 'Punto de partida', "Tiempo", "TipoRuta"] # Columnas que se consideran importantes.
df_rutas = adaptar_caracteristicas(rutas, "Ruta", columnas ,"Concejos", "texto") # No tienen coordenadas
df_rutas.head()


(0, 40)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url


Unir los dataframes al dataframe completo y guardarlo

In [36]:
df_definitivo = pd.concat([df_completo, df_bodegas, df_esp_nat, df_cultura, df_religioso, df_rutas], ignore_index= True) 
print(df_definitivo.shape)

df_definitivo.to_csv("../Data/Data_used/tourist_places_data.csv", index=False)

(1101, 8)


# Extra

Rutas en coche (¿podrían servir como ejemplos de itinerarios según los días?); areas de autocaravanas (¿podrían ser alojamientos?); oficinas de turismo (no sé si vale para algo); fiestas de interés turístico (¿se podrían añadir a lugares turísticos?)

In [37]:
rutas_coche = dfs["Rutas"][dfs["Rutas"]["TipoRuta"] == "coche"]
print(rutas_coche.shape)

columnas = ['Distancia', 'Dificultad', "Código", 'Tipo de recorrido', 'Altitud', 'Desnivel', 'Punto de partida', "TipoRuta"] # "Tiempo en coche"
df_rutas_coche = adaptar_caracteristicas(rutas_coche, "Ruta coche", columnas ,"Concejos", "texto") # No tienen coordenadas
df_rutas_coche["Días"] = rutas_coche["Tiempo en coche"]
df_rutas_coche.head()

(23, 40)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url,Días
4,El infinito encanto de Picos de Europa,"Amieva, Cangas de Onís, Onís, Cabrales y Peñam...",None,None,"[Distancia: 151,8, Dificultad: 1.0, Punto de p...",Un paraíso de montañas y de opciones para la a...,Ruta coche,None,5 días
15,"Oviedo/Uviéu, el Prerrománico y los Valles del...",Oviedo - Valles del oso,None,None,"[Distancia: 29, Dificultad: 1.0, Punto de part...",Oviedo/Uviéu es señorial. La historia de esta ...,Ruta coche,None,2 días
30,La cultura castreña y el Parque Histórico del ...,"Navia, Tapia de Casariego, Boal y Coaña",None,None,"[Distancia: 69, Dificultad: 1.0, Punto de part...",Con una superficie de setecientos setenta y si...,Ruta coche,None,5 días
35,El dorado asturiano,Tineo y Allande,None,None,"[Distancia: 90,1, Dificultad: 1.0, Punto de pa...",Son tierras de orografía agreste que adoptan f...,Ruta coche,None,2 días
55,Ruta de 7 días por el Patrimonio Industrial de...,"Villaviciosa, Gijón, Langreo, San Martín del R...",None,None,"[Punto de partida: Villaviciosa, TipoRuta: coche]","Estos emblemáticos elementos patrimoniales, mu...",Ruta coche,None,7 días


In [38]:
areas_auto = dfs["Areas_autocaravanas"]
print(areas_auto.shape)
columnas = ["Dirección", "Plazas", "Precio", "EstanciaMaxima", "Tipo", "Acceso", "Servicios", "Teléfono", "Email"]
df_auto = adaptar_caracteristicas(areas_auto, "Areas autocaravanas", columnas ,"Concejo", "descripcion")
df_auto.head()

(63, 21)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
0,Área autocaravanas Valgrande-Pajares,Lena,42.97889,-5.77194,"[Dirección: Brañillín. 33693. Pajares, Plazas:...",,Areas autocaravanas,None
1,Área autocaravanas de MasyMas (Cudillero),Cudillero,43.55528,-6.16917,"[Dirección: Las Dueñas, s/n 33159 Cudillero, P...",,Areas autocaravanas,None
2,Parking autocaravanas Valles del Oso (Tuñón),Santo Adriano,43.291667,-5.982778,"[Dirección: Ctra. AS228, km 8,500. 33115. Tuñó...",,Areas autocaravanas,None
3,Área autocaravanas de Santa Eulalia de Oscos,Santa Eulalia de Oscos,43.25778,-7.01861,[Dirección: C/ Vilar de Abraira. 33776. Santal...,,Areas autocaravanas,None
4,Área de Autocaravanas de Sellaño (Ponga),Ponga,43.25106,-5.19992,"[Dirección: Pilanegru s/n, Plazas: 12.0, Preci...",,Areas autocaravanas,None


In [39]:
oficinas = dfs["Oficinas_turismo"]
print(oficinas.shape)
columnas = ['Dirección', 'Código Postal', "Teléfono", 'Email', 'Web', "Facebook", "Twitter", "Instagram", 'Horario', 'Horario', 'Abierto todo el año']
df_oficinas = adaptar_caracteristicas(oficinas, "Oficina turismo", columnas ,"Localidad", "texto")
df_oficinas.head()

(67, 37)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
0,Oficina de turismo de Cabrales,Arenas,43.30318,-4.81751,"[Dirección: Plaza de los Cuatro Caños, Código ...",,Oficina turismo,None
1,Oficina de turismo de Siero,Pola de Siero,43.390088,-5.663658,[Dirección: Palacio Marqués de Santa Cruz. Alc...,,Oficina turismo,None
2,Oficina de turismo de Lena,La Pola,43.160709,-5.827519,"[Dirección: Marqués de San Feliz, 2, Código Po...",,Oficina turismo,None
3,Oficina de turismo de Cangas del Narcea,Cangas del Narcea,43.177134,-6.551716,"[Dirección: Palacio de Omaña. Plaza La Oliva, ...",,Oficina turismo,None
4,Punto de información turística en La Escalerona,Gijón/Xixón,43.542311,-5.659271,"[Dirección: Playa de San Lorenzo. Escalera 4, ...",,Oficina turismo,None


In [40]:
fiestas = dfs["Fiestas_interes_turistico"]
print(df_aux.shape)
columnas = ['Días', 'Email', 'Web', 'Facebook']
df_fiestas = adaptar_caracteristicas(fiestas, "Fiesta", columnas ,"Localidad", "descripcion")
df_fiestas.head()

(228, 40)


,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
0,San Timoteo,Luarca/lluarca,43.543920,-6.535111,[Días: 22 Agosto],"San Timoteo en Luarca/lluarca, con el chambrón...",Fiesta,None
1,Nuestra Señora del Carmen - La Descarga,Cangas del Narcea,43.175473,-6.550772,[Días: 16 Julio],El más grandioso homenaje a la Virgen en forma...,Fiesta,None
2,Fiesta de los Huevos Pintos,La Pola Siero,43.393427,-5.660276,"[Días: Martes de Pascua, Facebook: https://www...","Fiesta de los Huevos Pintos en La Pola Siero, ...",Fiesta,None
3,Sacramental de Cue,Cue,43.411693,-4.735728,[Días: 24 Junio],"O el arte por los suelos La localidad de Cue, ...",Fiesta,None
4,Fiesta de El Bollo,Avilés,43.553603,-5.920635,[Días: Lunes de Pascua],"Fiesta de El Bollo en Avilés, el más animado a...",Fiesta,None
